In [1]:


from api_connection import NotionAPIConnector
from api_result_fetcher import ResultFetcher
from updater import DictDequeueStructureUpdater
from interface_user import InterfaceUser

connector = NotionAPIConnector()
raw_response:dict = connector.main()
headers = connector.headers

fetcher = ResultFetcher(raw_response=raw_response, headers=headers)

res = fetcher.main()

In [3]:
updater = DictDequeueStructureUpdater(response_result_fetcher=res)

t = updater.update_and_save()
t

defaultdict(collections.deque,
            {'Ton per': deque([{'page ton père': 'https://www.notion.so/page-ton-p-re-50da195ac4d54448a0e255cbc9e65bb5'}]),
             'ta mere': deque([{'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}]),
             'Page test': deque([{'Page test': 'https://www.notion.so/Page-test-e54dd41c3ac247bbb7218cbb744990de'},
                    {'Salut les peds': 'https://www.notion.so/Salut-les-peds-a04171d69b3040dfbbd90e7577bcbfd3'},
                    {'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'},
                    {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'}])})

In [2]:
import datetime 

datetime.datetime.now().strftime('%H:%M:%S')

'22:23:12'

In [9]:
import pickle
import pathlib
from typing import Any


def load_pickle(path_object: pathlib.Path) -> Any:
    with open(path_object, "rb") as f:  # rb -> Read Binary
        return pickle.load(f)


def save_pickle(object_: Any, path_to_save: pathlib.Path) -> None:
    with open(path_to_save, "wb") as f:
        pickle.dump(object_, f)

t = load_pickle(r"\\wsl.localhost\Ubuntu\home\lucas\space-repetition-learning-with-notion\srl_notion\data\data_structure.pkl")

In [10]:
t

deque([{'L’effet “Matthew”': 'https://www.notion.so/L-effet-Matthew-bf96938692294aa4b3cba4b3189d1fbe'},
       {'Théorie': 'https://www.notion.so/Th-orie-6e47454e982041c6bcff0f470bd22620'},
       {'IE dans le cerveau': 'https://www.notion.so/IE-dans-le-cerveau-84dad139fab841ebb5c7b9f8e8c74382'},
       {'Les piliers de l’IE selon Goleman': 'https://www.notion.so/Les-piliers-de-l-IE-selon-Goleman-cd3d625b8f1e4bb89822ece8e3bd7020'},
       {'Emotional self awareness': 'https://www.notion.so/Emotional-self-awareness-2d57dba2fe9b40238c99061410c4b3a8'},
       {'Maitrise de soi': 'https://www.notion.so/Maitrise-de-soi-349fd4673392431aaa91d2322a6d4b53'},
       {'la colère': 'https://www.notion.so/la-col-re-46c08c7bc65a4c90be5c04b719b6fd94'},
       {'L’anxiété': 'https://www.notion.so/L-anxi-t-f8d61ddab4e6414397a87279e043684e'},
       {'Tristesse': 'https://www.notion.so/Tristesse-1fbef2d4855b4a7395cf2b397d08c4fe'},
       {'Repression': 'https://www.notion.so/Repression-0ec1c599d38344f4a